# Imports

In [91]:
import datetime
import pymongo
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import re
from pygeocoder import Geocoder, GeocoderError

import numpy as np
from collections import defaultdict

import plotly
import plotly.plotly as py
plotly.offline.init_notebook_mode(connected=True)

In [26]:
from sklearn import linear_model
import sklearn

# Connect to Data

##### Connect to databases

In [2]:
# Set up connection to mongodb
client = pymongo.MongoClient() # Connect to default client
db = client.TTB # Get a database (note: lazy evaluation)
TTB = db.TTB # collection for form data
TTB_COLORS = db.COLORS # collection for the label image data
TTB_IMG_META = db.IMG_META
TTB_IMG_SUP = db.IMG_SUP

##### Load into pandas

In [3]:
df = pd.DataFrame(list(TTB.find()))
df_colors = pd.DataFrame(list(TTB_COLORS.find()))
df_img_meta = pd.DataFrame(list(TTB_IMG_META.find()))
df_img_sup = pd.DataFrame(list(TTB_IMG_SUP.find()))

##### Drop useless `_id` columns

In [4]:
df_colors = df_colors.drop(['_id'], axis=1)
df_img_meta = df_img_meta.drop(['_id'], axis=1)
df_img_sup = df_img_sup.drop(['_id'], axis=1)

##### Drop duplicates, unclear why present

In [5]:
df_colors.shape[0] - df_colors.drop_duplicates().shape[0]

391

In [6]:
df_colors = df_colors.drop_duplicates()

In [6]:
df_img_meta.shape[0] - df_img_meta.drop_duplicates().shape[0]

119

In [7]:
df_img_meta = df_img_meta.drop_duplicates()

In [8]:
df_img_sup.shape[0] - df_img_meta.drop_duplicates().shape[0]

119

In [9]:
df_img_sup = df_img_sup.drop_duplicates()

##### Helper lookup for US states

In [10]:
# get list of all US states, convert to uppercase as that is what is used
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
states = [state.upper() for state in states]

us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}

# capitalized versions
abbrev_lookup=defaultdict(str)
for k, v in us_state_abbrev.items():
    abbrev_lookup[k.upper()] = v


##### Select US states only from the main df

In [11]:
us_only = df.loc[df['OriginCode'].isin(states)]
us_only = us_only.loc[df['Status'] == 'APPROVED']
us_only['_id'].count()

54484

# Analysis

## A quick look at our tables

In [13]:
df_img_meta.shape

(61465, 4)

In [14]:
df_img_meta.head()

,ImgType,LabelName,TTBID,URL
0,RGB,Brand (front),16001001000009,https://www.ttbonline.gov/colasonline/publicVi...
1,RGB,Brand (front),16001001000018,https://www.ttbonline.gov/colasonline/publicVi...
2,RGB,Back,16001001000018,https://www.ttbonline.gov/colasonline/publicVi...
3,RGB,Strip,16001001000018,https://www.ttbonline.gov/colasonline/publicVi...
4,RGB,Brand (front),16001001000021,https://www.ttbonline.gov/colasonline/publicVi...


In [15]:
df_colors.head()

,TTBID,b,g,img_num,percentage,r
0,16001001000009,57.854447,56.692722,0,0.50,60.746631
1,16001001000009,239.106518,238.742448,0,0.50,239.810811
2,16001001000018,166.517781,174.014225,0,0.50,179.844950
3,16001001000018,48.377104,50.838384,0,0.50,53.707071
4,16001001000018,232.319809,234.338902,1,0.25,236.873508


In [16]:
df_img_sup.head()

,EntropySum,TTBID
0,47094.762136,16001001000009
1,75152.047524,16001001000018
2,76623.535135,16001001000018
3,6882.318540,16001001000018
4,75163.828744,16001001000021


## Where on the packaging do these labels go?

Almost all labels are simpley pasted on the front, or back, though there are some that go elsewhere

In [12]:
df_img_meta.groupby('LabelName').count()

,ImgType,TTBID,URL
LabelName,,,
Back,19443,19443,19443
Brand (front),37652,37652,37652
Neck,1746,1746,1746
Other,1959,1959,1959
Strip,665,665,665


## What types of images are we getting (can we do anything with the alpha?)

Join our tables, to get a new table with the recieve date and the type of image

In [13]:
alpha_date = pd.merge(df[['recieve_date', 'TTBID']], 
                      df_img_meta[['TTBID', 'ImgType']], 
                      left_on='TTBID', right_on='TTBID', how='inner')

In [14]:
alpha_date['recieve_date'] = pd.to_datetime(alpha_date['recieve_date'], format='%m/%d/%Y')

In [15]:
alpha_date.head()

,recieve_date,TTBID,ImgType
0,2016-01-01,16001001000009,RGB
1,2016-01-01,16001001000018,RGB
2,2016-01-01,16001001000018,RGB
3,2016-01-01,16001001000018,RGB
4,2016-01-01,16001001000021,RGB


We can see how the applications evolve over time by using Grouper to look at weekly changes (`W`) or montly changes (`M`), and also grouping by the image type

In [48]:
img_type_by_date = alpha_date.groupby([pd.Grouper(key='recieve_date', freq='W'), 'ImgType']).count()
img_type_by_date.columns = ['counts']

In [49]:
img_type_by_date = img_type_by_date.unstack()
img_type_by_date.head()

counts           
ImgType        GREY   RGB RGBA
recieve_date                  
2016-01-03        1    96   16
2016-01-10       26  1720   97
2016-01-17       43  1615   92
2016-01-24       33  1553   69
2016-01-31       21  1545   80

In [50]:
img_type_by_date.columns

MultiIndex(levels=[['counts'], ['GREY', 'RGB', 'RGBA']],
           labels=[[0, 0, 0], [0, 1, 2]],
           names=[None, 'ImgType'])

In [54]:
greys = dict(
            type='scatter',
            name='Grey',
            x=img_type_by_date.index,
            y=img_type_by_date['counts']['GREY'])
rgb = dict(
            type='scatter',
            name='RGB',
            x=img_type_by_date.index,
            y=img_type_by_date['counts']['RGB'])
rgba = dict(
            type='scatter',
            name='RGBA',
            x=img_type_by_date.index,
            y=img_type_by_date['counts']['RGBA'])

layout = dict(
            title = 'Image Type By Time',
            xaxis = dict(title='Week', tickangle = -45),
            yaxis = dict(title='Number of Applications')
             )

# clean version, no labels
layout = dict(
            xaxis = dict(visible = False),
            yaxis = dict(visible = False)
             )

fig = dict( data=[greys, rgb, rgba], layout=layout )
plotly.offline.iplot( fig )

In [53]:
plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

'<div id="a4d05b26-b670-4d41-9ef4-12df6cbc1e87" style="height: 100%; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("a4d05b26-b670-4d41-9ef4-12df6cbc1e87", [{"type": "scatter", "name": "Grey", "x": ["2016-01-03", "2016-01-10", "2016-01-17", "2016-01-24", "2016-01-31", "2016-02-07", "2016-02-14", "2016-02-21", "2016-02-28", "2016-03-06", "2016-03-13", "2016-03-20", "2016-03-27", "2016-04-03", "2016-04-10", "2016-04-17", "2016-04-24", "2016-05-01", "2016-05-08", "2016-05-15", "2016-05-22", "2016-05-29", "2016-06-05", "2016-06-12", "2016-06-19", "2016-06-26", "2016-07-03", "2016-07-10", "2016-07-17", "2016-07-24", "2016-07-31", "2016-08-07", "2016-08-14", "2016-08-21", "2016-08-28", "2016-09-04", "2016-09-11", "2016-09-18", "2016-09-25", "2016-10-02", "2016-10-09", "2016-10-16", "2016-10-23", "2016-10-30", "2016-11-06", "2016-11-13", "2016-11-20", "2016-11-27", "

#### It's worth noting that although there is little change in the number of alpha images, if they are manufactured in large quantities, it may still be of interest to ACTEGA

In [57]:
a_only = alpha_date.loc[alpha_date['ImgType'] == 'RGBA']
a_only.shape

(3674, 3)

In [58]:
a_companies = pd.merge(a_only, df, left_on='TTBID', right_on='TTBID', how='inner')

Although there were close to 3700 new applications that included images with _clear_ parts, these came from a small subset of companies

In [61]:
a_companies['PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)'].unique().shape

(930,)

In [75]:
addresses = a_companies['PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)'].unique()

In [76]:
addresses = [re.split(r'\n', line) for line in addresses]
addresses = [list(filter(None, address)) for address in addresses]

In [77]:
addresses[1]

['DSP-KY-15008',
 'THREE SPRINGS BOTTLING COMPANY, LLC',
 '201 CAMPING WORLD CT',
 'BOWLING GREEN, KY 42104']

In [87]:
test = ''.join(addresses[0][2:])
print(test)
res = Geocoder.geocode(test)
res.valid_address

4545 FELTERRDMILPITAS, CA 95035


In [118]:
lat_long_list = []
for address in addresses:
    try:
        res = Geocoder.geocode(address[2:])
        if res.valid_address:
            lat_long_list.append((address[1], res.latitude, res.longitude))
    except GeocoderError:
        print(address)

['BWN-CA-15202', 'BIG DOG VINEYARDS, HOWELL CORP., INC.', '4545 FELTERRD', 'MILPITAS, CA 95035']
['DSP-KY-15008', 'THREE SPRINGS BOTTLING COMPANY, LLC', '201 CAMPING WORLD CT', 'BOWLING GREEN, KY 42104']
['BR-NY-21136', 'HEIKO GIESBERG', '7174ROUTE 209, HEIKO', 'WAWARSING, NY 12489']
['BR-NM-21014', 'TRACTOR BREWING COMPANY, TROUBLED MINDS, INC.', '18004TH ST, NW', 'ALBUQUERQUE, NM 87102']
['DSP-WA-21003', 'HERITAGE DISTILLING, HERITAGE DISTILLING COMPANY, INC.', '320757TH CT NW', 'GIG HARBOR, WA 98335']
['BWN-PA-21014', 'THE COLONY MEADERY, LLC', '905HARRISON ST , STE 115', 'ALLENTOWN, PA 18105']
['BWN-CA-21397', 'NORTH AMERICAN DIRT FARMER, JASON EDWARD CHARLES, LLC', '216608TH ST, E', 'SONOMA, CA 95476']
['DSP-KY-15007', 'STRONG SPIRITS, INC.', '999 WITHROW CT', 'BARDSTOWN, KY 40004']
['BR-WA-IRO-15002', 'IRON GOAT BREWING, IRON GOAT LLC', '2204 E MALLON AVE , B', 'SPOKANE, WA 99202']
['BWN-CA-15060', 'CUVAISON WINERY, CUVAISON, INC.', '1221 DUHIG RD', 'NAPA, CA 94559']
['DSP-NH-150

['BWN-CA-16663', 'SINGER CELLARS, BARRY SINGER GALLERY,INC', '833 CHILENO VALLEY RD', 'PETALUMA, CA 94952']
['DSP-NY-21071', 'THIEVES SPIRITS, BITTER THIEVES, LLC', '630FLUSHING AVE', 'BROOKLYN, NY 11206']
['BWN-SC-21011', 'LAZY BEAR WINERY, LAZY BEAR WINERY LLC', '644HESTER STORE RD', 'EASLEY, SC 29640']
['BWN-CA-15812', 'JOSEPH PHELPS VINEYARDS LLC', '1625 FREESTONE FLAT RD', 'FREESTONE, CA 95472']
['BWN-WV-15012', 'MOSS FARMS WINERY, LLC', '100 HARVEST LN', 'WHEELING, WV 26003']
['BWN-CA-16153', 'C2 CELLARS, C2 CELLARS, INC.', '9050 SAN ANTONIORD', 'ORCUTT, CA 93455']
['BW-OR-320', 'NW WINE COMPANY, LLC', '20980 NE NIEDERBERGER RD', 'DUNDEE, OR 97115']
['BR-OR-WIL-15001', '10 BARREL BREWING, 10 BARREL BREWING, LLC', '62970 18TH ST', 'BEND, OR 97701']
['BR-NC-HAM-15000', "NATTY GREENE'S BREWING COMPANY, HAMBURGER SQUARE BREWHOUSE, INC.", '1918 W LEE ST', 'GREENSBORO, NC 27403']
['BW-CA-5368', 'CHATEAU DIANA, CHATEAU DIANA, LLC', '6195 DRY CREEK RD', 'HEALDSBURG, CA 95448']
['BWN-CA-2

['BWN-CA-15127', 'D.R. STEPHENS ESTATE WINES, D.R. STEPHENS ESTATE WINES, LLC', '3524SILVERADO TRL', 'SAINT HELENA, CA 94574']
['BW-CA-5398', 'BEAULIEU VINEYARD, TREASURY WINE ESTATES AMERICAS COMPANY', '2146821480, 216728TH STREET EAST', 'SONOMA, CA 95476']
['BWN-NY-21039', 'APPLE COUNTRY SPIRITS, CELK DISTILLING, LLC', '3274EDDY RD', 'WILLIAMSON, NY 14589']
['DSP-WV-21006', 'MOSS FARMS WINERY, LLC', '100HARVEST LN', 'WHEELING, WV 26003']
['DSP-MN-20003', 'CROOKED WATER, NORSEMAN DISTILLERY LLC', '429TAFT ST NE STE 19', 'MINNEAPOLIS, MN 55413']
['BR-NC-21141', 'CABARRUS BREWING COMPANY, LLC', '329MCGILL AVE NW', 'CONCORD, NC 28027']
['BR-DE-MIL-15000', '16 MILE, 16 MILE BREWING COMPANY INC', '413 S BEDFORD ST', 'GEORGETOWN, DE 19947']
['BWN-OR-15195', 'ABIQUA WIND VINEYARD, LGA, INC.', '19822MCKILLOP RD', 'SCOTTS MILLS, OR 97375']
['BWN-OH-15123', 'TNT WINERY LLC', '1165GRANADA RD NW', 'DELLROY, OH 44620']
['BR-PA-VIC-1', 'VICTORY BREWING COMPANY', '420ACORN LN', 'DOWNINGTOWN, PA 1933

['BR-OR-21019', 'PLANK TOWN BREWING CO., PLANKTOWN LLC', '346MAIN ST', 'SPRINGFIELD, OR 97477']
['BR-CA-MAR-15001', 'OCEANSIDE ALE WORKS, MARK ROBIN PURCIEL & SCOTT TAYLOR THOMAS "PARTNERSHIP"', '1800ORD WAY', 'OCEANSIDE, CA 92056']
['BWN-WA-21053', 'HARMELL CELLARS, LLC', '269NARCISSA PL', 'WALLA WALLA, WA 99362']
['BR-IA-20051', 'KEG CREEK BREWING COMPANY, LLC', '22381221ST ST', 'GLENWOOD, IA 51534']
['BWN-CA-21943', 'FILLMORE WEST VINEYARDS, FILLMORE WEST VINEYARDS, LLC', '2455W 3RD ST', 'SAN FRANCISCO, CA 94107']
['BWN-CA-21723', 'CORNER 103, LLC', '214818TH STSTE 30', 'SONOMA, CA 95476']
['BW-WA-316', 'SYNCLINE WINE CELLARS, MORCHELLA WINE CELLARS, LLC', '111BALCH RD', 'LYLE, WA 98635']
['BWN-VA-15055', 'SUNSET HILLS VINEYARD, LLC', '38295FREMONT OVERLOOK LN', 'PURCELLVILLE, VA 20132']
['BR-PA-VIC-1', 'VICTORY BREWING COMPANY, LLC', '420ACORN LN', 'DOWNINGTOWN, PA 19335']
['DSP-MI-20064', 'HOMES BREWERY LLC', '2321JACKSON AVE', 'ANN ARBOR, MI 48103']
['DSP-NY-15056', 'SOUTHERN TIE

In [137]:
lat_long = pd.DataFrame(lat_long_list)
lat_long.columns = ['name', 'lat', 'long']
lat_long.shape

(314, 3)

In [139]:
res = a_companies[['PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)', 'TTBID']].groupby(
    'PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)').count()
res.reset_index(inplace=True)
res['name'] = [re.split(r'\n', registry)[1] for registry in res['PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)']]
res.shape

(930, 3)

In [140]:
lat_long = pd.merge(lat_long, res, left_on='name', right_on='name', how='inner')
lat_long.drop('PlantRegistry/BasicPermit/BrewersNo(PrincipalPlaceofBusiness)', 1, inplace=True)
lat_long.rename(columns={'TTBID': 'count'}, inplace=True)
lat_long.head()

,name,lat,long,count
0,"RIVER HORSE BREWING COMPANY, RHB ACQUISITION LLC",40.276377,-74.796004,2
1,"RIVER HORSE BREWING COMPANY, RHB ACQUISITION LLC",40.276377,-74.796004,4
2,"The Flag Ship Brewing Company, GORDON JAMES LLC",40.636929,-74.075816,2
3,"Day Wines, DAY CRUSH, LLC",45.267059,-123.019978,1
4,"SOKOL BLOSSER WINERY, SOKOL BLOSSER LTD",45.251523,-123.049899,11


In [161]:
locations = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            name = 'Locations',
            lat=lat_long['lat'],
            lon = lat_long['long'],
            text = lat_long['name'] + ':  ' + lat_long['count'].astype(str),
            mode = 'markers',
            marker = dict(size=5,
                         color = lat_long['count'],
                         cmax = lat_long['count'].max(),
                         colorbar=dict(title="")
                         )
            )


# clean version, no labels
layout = dict(
            title = 'Manufacturers using alpha',
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = "rgb(125, 125, 125)",
                subunitcolor = "rgb(217, 217, 217)",
                countrycolor = "rgb(217, 217, 217)",
                countrywidth = 0.5,
                subunitwidth = 0.5
            ),
             )

fig = dict( data=[locations], layout=layout )
plotly.offline.iplot( fig )

In [162]:
plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

'<div id="43b0a6b1-3650-4893-b1bd-c39f3b68bd6b" style="height: 100%; width: 100%;" class="plotly-graph-div"></div><script type="text/javascript">window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("43b0a6b1-3650-4893-b1bd-c39f3b68bd6b", [{"type": "scattergeo", "locationmode": "USA-states", "name": "Locations", "lat": [40.2763769, 40.2763769, 40.6369293, 45.267059, 45.2515229, 45.2515229, 38.2220226, 37.6642676, 40.7057895, 36.90563059999999, 39.113784, 44.9428926, 35.69956, 30.280047, 37.7256219, 47.5676354, 41.4740236, 39.9533889, 38.9717749, 38.105871, 38.650524, 35.8087141, 44.8442274, 37.7801683, 40.404099, 32.654689, 38.4360065, 38.4360065, 41.4395701, 38.5366485, 42.50398999999999, 41.2377324, 46.53825699999999, 38.6456653, 41.8776516, 38.93631269999999, 44.5049225, 42.9201128, 41.1890496, 45.5191382, 38.4791219, 38.9791301, 38.22036689999999, 45.30221969999999, 39.8756046, 33.729281, 38.4791219, 38.4446747, 44.8439878, 41.8816597, 3

## Number of colors by type of beverage

We can group more broadly using the following categories

In [23]:
SPIRITS_MISC = ["CORDIALS (FRUIT & PEELS)", "FRUIT FLAVORED LIQUEURS", "CURACAO", "TRIPLE SEC", "FRUITS & PEELS SCHNAPPS LIQUEUR", "OTHER FRUITS & PEELS LIQUEURS", "CORDIALS (HERBS & SEEDS)", "ANISETTE, OUZO, OJEN", "COFFEE (CAFE) LIQUEUR", "KUMMEL", "PEPPERMINT SCHNAPPS", "HERBS AND SEEDS SCHNAPPS LIQUEUR", "AMARETTO", "SAMBUCA", "ARACK/RAKI/ARAK", "OTHER (HERBS & SEEDS)", "CORDIALS (CREMES OR CREAMS)", "CREME DE CACAO WHITE", "CREME DE CACAO BROWN", "CREME DE MENTHE WHITE", "CREME DE MENTHE GREEN", "CREME DE ALMOND (NOYAUX)", "DAIRY CREAM LIQUEUR/CORDIAL", "NON DAIRY CREME LIQUEUR/CORDIAL", "OTHER LIQUEUR (CREME OR CREAMS)", "SPECIALTIES & PROPRIETARIES", "OTHER SPECIALTIES & PROPRIETARIES", "CORDIALS (FRUIT & PEELS)", "FRUIT FLAVORED LIQUEURS", "CURACAO", "TRIPLE SEC", "FRUITS & PEELS SCHNAPPS LIQUEUR", "OTHER FRUIT & PEELS LIQUEURS", "CORDIALS (HERBS & SEEDS)", "ANISETTE, OUZO, OJEN", "COFFEE (CAFE) LIQUEUR", "KUMMEL", "PEPPERMINT SCHNAPPS", "HERBS & SEEDS SCHNAPPS LIQUEUR", "AMARETTO", "SAMBUCA", "ARACK/RAKI/ARAK", "OTHER HERB & SEED CORDIALS/LIQUEURS", "CORDIALS (CREMES OR CREAMS)", "CREME DE CACAO WHITE", "CREME DE CACAO BROWN", "CREME DE MENTHE WHITE", "CREME DE MENTHE GREEN", "CREME DE ALMOND (NOYAUX)a", "DAIRY CREAM LIQUEUR/CO", "NON DAIRY CREME LIQUEUR/CORDIAL", "OTHER LIQUEUR (CREMES OR CREAMS)", "SPECIALITIES & PROPRIETARIES", "OTHER SPECIALTIES & PROPRIETARIES", "COCKTAILS 48 PROOF UP", "MARGARITA (48 PROOF UP)", "COCKTAILS 48 PROOF UP (CONT)", "DAIQUIRI (48 PROOF UP)", "COLADA (48PROOF UP)", "OTHER COCTAILS (48PROOF UP)", "COCKTAILS UNDER 48 PROOF", "MARGARITA (UNDER 48 PROOF)", "OTHER TEQUILA-BASED COCKTAILS (UNDER 48 PROOF)", "COCKTAILS UNDER 48 PROOF (CONT)", "DAIQUIRI (UNDER 48 PROOF)", "COLADA (UNDER 48 PROOF)", "OTHER COCKTAILS (UNDER 48 PROOF)", "MIXED DRINKS-HI BALLS COCKTAILS", "SCREW DRIVER", "COLLINS", "BLOODY MARY", "EGG NOG", "OTHER MIXED DRINKS HI-BALLS COCKTAILS", "COCKTAILS 48 PROOF UP", "MARGARITA 48 PROOF UP", "DAIQUIRI 48 PROOF UP", "COLADA (48 PROOF UP )", "OTHER COCKTAILS (48 PROOF UP)", "COCKTAILS UNDER 48 PROOF", "MARGARITA UNDER 48 PROOF", "OTHER TEQUILA-BASED COCKTAILS (UNDER 48 PROOF)", "COCKTAILS UNDER 48 PR(CONT)", "DAIQUIRI UNDER 48 PROOF", "COLADA (UNDER 48 PROOF )", "OTHER COCKTAILS (UNDER 48 PROOF)", "MIXED DRINKS-HI BALLS COCKTAILS", "SCREW DRIVER", "COLLINS", "BLOODY MARY", "EGG NOG", "OTHER MIXED DRINKS HI-BALLS COCKTAILS", "86	VERMOUTH/MIXED TYPES", "OTHER SPIRITS", "NEUTRAL SPIRITS - GRAIN", "NEUTRAL SPIRITS - FRUIT", "NEUTRAL SPIRITS - CANE", "NEUTRAL SPIRITS - VEGETABLE", "NEUTRAL SPIRITS - PETROLEUM", "BITTERS - BEVERAGE", "GRAIN SPIRITS", "OTHER SPIRITS", "SAKE", "SAKE - DOMESTIC FLAVORED", "NON ALCOHOLIC MIXES", "OTHER SPIRITS", "NEUTRAL SPIRITS - GRAIN", "NEUTRAL SPIRITS - FRUIT", "NEUTRAL SPIRITS - CANE", "NEUTRAL SPIRITS - VEGETABLE", "NEUTRAL SPIRITS - PETROLEUM", "BITTERS - BEVERAGE*", "TEQUILA FB", "TEQUILA USB", "MEZCAL", "MEZCAL FB", "SAKE - IMPORTED", "MEZCAL US", "SAKE - IMPORTED FLAVORED", "DILUTED TEQUILA FB", "DILUTED TEQUILA USB", "NON ALCOHOL MIXES"]
BEERS = ["BEER", "CEREAL BEVERAGES - NEAR BEER (NON ALCOHOLIC)", "OTHER MALT BEVERAGES (BEER)", "BEER", "CEREAL BEVERAGES - NEAR BEER (NON ALCOHOLIC)", "MALT BEVERAGES", "ALE", "MALT LIQUOR", "STOUT", "PORTER", "MALT BEVERAGES SPECIALITIES - FLAVORED", "MALT BEVERAGES SPECIALITIES", "OTHER MALT BEVERAGES"]
WINE = ["TABLE RED WINE", "ROSE WINE", "TABLE WHITE WINE", "TABLE FLAVORED WINE", "TABLE FRUIT WINE", "SPARKLING WINE/CHAMPAGNE", "CARBONATED WINE", "DESSERT FLAVORED WINE", "DESSERT /PORT/SHERRY/(COOKING) WINE", "DESSERT FRUIT WINE"]
BRANDY = ["CALIFORNIA GRAPE BRANDY", "CALIFORNIA BRANDY", "CALIFORNIA DRIED BRANDY", "CALIFORNIA LEES BRANDY", "CALIFORNIA POMACE OR MARC BRANDY", "CALIFORNIA RESIDUE BRANDY", "CALIFORNIA NEUTRAL BRANDY", "OTHER CALIFORNIA BRANDY", "NEW YORK GRAPE BRANDY", "NEW YORK BRANDY", "NEW YORK DRIED BRANDY", "NEW YORK LEES BRANDY", "NEW YORK POMACE OR MARC BRANDY", "NEW YORK RESIDUE BRANDY", "NEW YORK NEUTRAL BRANDY", "OTHER NEW YORK BRANDY", "OTHER DOMESTIC GRAPE BRANDY", "BRANDY", "DRIED BRANDY", "LEES BRANDY", "POMACE OR MARC BRANDY", "RESIDUE BRANDY", "NEUTRAL BRANDY", "IMMATURE BRANDY", "OTHER BRANDY", "BRANDY - FLAVORED", "BRANDY - APRICOT FLAVORED", "BRANDY - BLACKBERRY FLAVORED", "BRANDY - PEACH FLAVORED", "BRANDY - CHERRY FLAVORED", "BRANDY - COFFEE FLAVORED", "BRANDY APPLE FLAVORED", "OTHER BRANDY - FLAVORED", "FRUIT BRANDY", "APPLE BRANDY", "CHERRY BRANDY", "PLUM BRANDY", "BLACKBERRY BRANDY", "BLENDED APPLE JACK BRANDY", "PEAR BRANDY", "APRICOT BRANDY", "DILUTED BRANDY", "OTHER FRUIT BRANDY", "FRENCH BRANDY", "COGNAC (BRANDY) FB", "COGNAC (BRANDY) USB", "ARMAGNAC (BRANDY) FB", "ARMAGNAC (BRANDY) USB", "OTHER FRENCH BRANDY FB", "OTHER FRENCH BRANDY USB", "OTHER FOREIGN BRANDY", "ITALIAN GRAPE BRANDY FB", "ITALIAN GRAPE BRANDY USB", "SPANISH GRAPE BRANDY FB", "SPANISH GRAPE BRANDY USB", "PORTUGUESE GRAPE BRANDY FB", "PORTUGUESE GRAPE BRANDY USB", "GREEK GRAPE BRANDY FB", "GREEK GRAPE BRANDY USB", "GERMAN GRAPE BRANDY FB", "GERMAN GRAPE BRANDY USB", "AUSTRALIAN GRAPE BRANDY FB", "AUSTRALIAN GRAPE BRANDY USB", "SOUTH AFRICAN GRAPE BRANDY FB", "SOUTH AFRICAN GRAPE BRANDY USB", "OTHER FOREIGN BRANDY (CONT.)", "BRANDY APRICOT FLAVORED", "BRANDY BLACKBERRY FLAVORED", "BRANDY PEACH FLAVORED", "BRANDY CHERRY FLAVORED", "BRANDY COFFEE FLAVORED", "BRANDY APPLE FLAVORED", "OTHER GRAPE BRANDY (PISCO, GRAPPA) FB", "OTHER GRAPE BRANDY (GRAPPA) USB", "FOREIGN FRUIT BRANDY", "APPLE BRANDY (CALVADOS)", "CHERRY BRANDY", "PLUM BRANDY (SLIVOVITZ)", "BLACKBERRY BRANDY", "BLENDED APPLE JACK BRANDY", "APRICOT BRANDY", "DILUTED BRANDY FB", "DILUTED BRANDY USB", "OTHER FRUIT BRANDY", "BLACKBERRY FLAVORED BRANDY", "LIQUEUR & BRANDY", "FLAVORED BRANDY", "APRICOT FLAVORED BRANDY", "BLACKBERRY FLAVORED BRANDY", "PEACH FLAVORED BRANDY", "CHERRY FLAVORED BRANDY", "LIQUEUR & BRANDY", "OTHER FLAVORED BRANDY", "BRANDY STINGER (48 PROOF UP)", "BRANDY SIDE CAR (48 PROOF UP)", "BRANDY STINGER (UNDER 48 PROOF)", "BRANDY SIDE CAR (UNDER 48 PROOF)", "BRANDY STINGER 48 PROOF UP", "BRANDY SIDE CAR 48 PROOF UP", "BRANDY STINGER UNDER 48 PROOF", "BRANDY SIDE CAR UNDER 48 PROOF", "BRANDY PEAR FLAVORED"]
GIN = ["DISTILLED GIN", "LONDON DRY DISTILLED GIN", "OTHER DISTILLED GIN", "GIN", "LONDON DRY GIN", "OTHER GIN", "GIN - FLAVORED", "GIN - MINT FLAVORED", "GIN - ORANGE FLAVORED", "GIN - LEMON FLAVORED", "OTHER GIN - FLAVORED", "DILUTED GIN", "DISTILLED GIN", "LONDON DRY DISTILLED GIN FB", "LONDON DRY DISTILLED GIN USB", "OTHER DISTILLED GIN FB", "OTHER DISTILLED GIN USB", "GIN", "LONDON DRY GIN FB", "LONDON DRY GIN USB", "OTHER GIN FB", "OTHER GIN USB", "GIN - FLAVORED", "GIN - MINT FLAVORED", "GIN - ORANGE FLAVORED", "GIN - LEMON FLAVORED", "GIN - CHERRY FLAVORED", "GIN - APPLE FLAVORED", "GIN - BLACKBERRY FLAVORED", "GIN - PEACH FLAVORED", "GIN - GRAPE FLAVORED", "OTHER GIN - FLAVORED", "OTHER GIN", "DILUTED GIN FB", "DILUTED GIN USB", "BRANDY - GINGER FLAVORED", "BRANDY GINGER FLAVORED", "SLOE GIN", "GIN SPECIALTIES", "LIQUEURS (GIN)", "SLOE GIN", "GINGER FLAVORED BRANDY", "GIN SPECIALTIES", "LIQUEURS (GIN)", "GIN MARTINI (48 PROOF UP)", "GIN SOUR (48 PROOF UP)", "GIN MARTINI (UNDER 48 PROOF)", "GIN SOUR (UNDER 48 PROOF)", "GIN MARTINI 48 PROOF UP", "GIN SOUR 48 PROOF UP", "GIN MARTINI UNDER 48 PROOF", "GIN SOUR UNDER 48 PROOF"]
WHISKY = ["STRAIGHT WHISKY", "STRAIGHT BOURBON WHISKY", "STRAIGHT RYE WHISKY", "STRAIGHT CORN WHISKY", "OTHER STRAIGHT WHISKY", "WHISKY BOTTLED IN BOND (BIB)", "BOURBON WHISKY BIB", "RYE WHISKY BIB", "CORN WHISKY BIB", "OTHER WHISKY BIB", "STRAIGHT WHISKY BLENDS", "STRAIGHT BOURBON WHISKY BLENDS", "STRAIGHT RYE WHISKY BLENDS", "STRAIGHT CORN WHISKY BLENDS", "OTHER STRAIGHT BLENDED WHISKY", "WHISKY BLENDS", "BLENDED BOURBON WHISKY", "BLENDED RYE WHISKY", "BLENDED CORN WHISKY", "BLENDED LIGHT WHISKY", "BLENDED WHISKY", "DILUTED BLENDED WHISKY", "OTHER WHISKY BLENDS", "WHISKY", "BOURBON WHISKY", "RYE WHISKY", "CORN WHISKY", "LIGHT WHISKY", "WHISKY PROPRIETARY", "SPIRIT WHISKY", "DILUTED WHISKY", "OTHER WHISKY (FLAVORED)", "SCOTCH WHISKY", "SCOTCH WHISKY FB", "SCOTCH WHISKY USB", "SINGLE MALT SCOTCH WHISKY", "UNBLENDED SCOTCH WHISKY USB", "DILUTED SCOTCH WHISKY FB", "DILUTED SCOTCH WHISKY USB", "CANADIAN WHISKY", "CANADIAN WHISKY FB", "CANADIAN WHISKY USB", "DILUTED CANADIAN WHISKY FB", "DILUTED CANADIAN WHISKY USB", "IRISH WHISKY", "IRISH WHISKY FB", "IRISH WHISKY USB", "DILUTED IRISH WHISKY FB", "DILUTED IRISH WHISKY USB", "WHISKY ORANGE FLAVORED", "WHISKY GRAPE FLAVORED", "WHISKY LIME FLAVORED", "WHISKY LEMON FLAVORED", "WHISKY CHERRY FLAVORED", "WHISKY CHOCOLATE FLAVORED", "WHISKY MINT FLAVORED", "WHISKY PEPPERMINT FLAVORED", "WHISKY PTHER FLAVORED", "OTHER IMPORTED WHISKY", "OTHER IMPORTED WHISKY FB", "OTHER IMPORTED WHISKY USB", "DILUTED OTHER IMPORTED WHISKY FB", "DILUTED OTHER IMPORTED WHISKY USB", "WHISKY SPECIALTIES", "LIQUEURS (WHISKY)", "WHISKY SPECIALTIES", "LIQUEURS (WHISKY)", "WHISKY MANHATTAN (48 PROOF UP)", "WHISKY OLD FASHIONED (48 PROOF UP)", "WHISKY SOUR (48 PROOF UP )", "WHISKY MANHATTAN (UNDER 48 PROOF)", "WHISKY OLD FASHIONED (UNDER 48 PROOF)", "WHISKY SOUR (UNDER 48 PROOF)", "WHISKY MANHATTAN (48 PROOF UP)", "WHISKY OLD FASHIONED (48 PROOF UP)", "WHISKY SOUR (48 PROOF UP)", "WHISKY MANHATTAN UNDER 48 PROOF", "WHISKY OLD FASHIONED UNDER 48 PROOF", "WHISKY SOUR UNDER 48 PROOF"]
RUM = ["RUM LEMON FLAVORED", "RUM CHERRY FLAVORED", "RUM CHOCOLATE FLAVORED", "RUM MINT FLAVORED", "RUM PEPPERMINT FLAVORED", "UR.S. RUM (WHITE)", "PUERTO RICAN RUM (WHITE)", "VIRGIN ISLANDS RUM (WHITE)", "HAWAIIAN RUM (WHITE)", "FLORIDA RUM (WHITE)", "OTHER RUM (WHITE)", "U.S. RUM (GOLD)", "PUERTO RICAN RUM (GOLD)", "VIRGIN ISLAND RUM (GOLD)", "HAWAIIAN RUM (GOLD)", "FLORIDA RUM (GOLD)", "OTHER RUM (GOLD)", "RUM FLAVORED (BOLD)", "RUM ORANGE GLAVORED", "RUM GRAPE FLAVORED", "RUM LIME FLAVORED", "RUM LEMON FLAVORED", "RUM CHERRY FLAVORED", "RUM CHOCOLATE FLAVORED", "RUM MINT FLAVORED", "RUM PEPPERMINT FLAVORED", "RUM OTHER FLAVORED", "OTHER WHITE RUM", "FLAVORED RUM (BOLD)", "RUM ORANGE FLAVORED", "RUM GRAPE FLAVORED", "RUM LIME FLAVORED", "RUM OTHER FLAVORED", "DILUTED RUM (WHITE)", "DILUTED RUM (GOLD)", "DOMESTIC FLAVORED RUM", "FOREIGN RUM", "CUBAN RUM WHITE FB", "CUBAN RUM WHITE USB", "CUBAN RUM GOLD FB", "CUBAN RUM GOLD USB", "OTHER FORIEGN RUM", "JAMAICAN RUM WHITE FB", "JAMAICAN RUM WHITE USB", "JAMAICAN RUM GOLD FB", "JAMICAN RUM GOLD USB", "GUIANAN RUM WHITE FB", "GUIANAN RUM WHITE USB", "DUTCH GUIANAN RUM GOLD FB", "DUTCH GUIANAN RUM GOLD USB", "FRENCH GUIANAN RUM FB", "FRENCH GUIANAN RUM USB", "MARTINICAN RUM WHITE FB", "MARTINICAN RUM WHITE USB", "MARTINICAN RUM GOLD FB", "MARTINICAN RUM GOLD USB", "OTHER FOREIGN RUM", "OTHER RUM WHITE FB", "OTHER RUM WHITE USB", "OTHER RUM GOLD FB", "OTHER RUM GOLD USB", "DILUTED RUM WHITE FB", "DILUTED RUM WHITE USB", "DILUTED RUM GOLD FB", "DILUTED RUM GOLD USB", "IMPORTED FLAVORED RUM", "ROCK & RYE, RUM & BRANDY (ETC.)", "RUM SPECIALTIES", "LIQUEURS (RUM)", "ROCK & RYE, RUM & BRANDY (ETC)", "RUM SPECIALTIES", "LIQUEURS (RUM)"]
VODKA = ["VODKA", "VODKA 80-90 PROOF", "VODKA", "VODKA 90-99 PROOF", "VODKA", "VODKA 100 PROOF UP", "VODKA - FLAVORED", "VODKA - ORANGE FLAVORED", "VODKA - GRAPE FLAVORED", "VODKA - LIME FLAVORED", "VODKA - LEMON FLAVORED", "VODKA - CHERRY FLAVORED", "VODKA - CHOCOLATE FLAVORED", "VODKA - MINT FLAVORED", "VODKA - PEPPERMINT FLAVORED", "VODKA - OTHER FLAVORED", "OTHER VODKA", "DILUTED VODKA", "VODKA", "VODKA 80-90 PROOF FB", "VODKA 80-90 PROOF USB", "VODKA", "VODKA 90-99 PROOF FB", "VODKA 90-99 PROOF USB", "VODKA", "VODKA 100 PROOF UP FB", "VODKA 100 PROOF UP USB", "VODKA - FLAVORED", "VODKA - ORANGE FLAVORED", "VODKA - GRAPE FLAVORED", "VODKA - LIME FLAVORED", "VODKA - LEMON FLAVORED", "VODKA - CHERRY FLAVORED", "VODKA - CHOCOLATE FLAVORED", "VODKA - MINT FLAVORED", "VODKA - PEPPERMINT FLAVORED", "VODKA - OTHER FLAVORED", "OTHER VODKA", "DILUTED VODKA FB", "DILUTED VODKA USB", "VODKA SPECIALTIES", "LIQUEURS (VODKA)", "VODKA SPECIALTIES", "LIQUEURS (VODKA)", "VODKA MARTINI (48 PROOF UP)", "VODKA MARTINI (UNDER 48 PROOF)", "VODKA MARTINI 48 PROOF UP", "VODKA MARTINI UNDER 48 PROOF", "VODKA SOUR UNDER 48 PROOF"]

##### Count the number of dominant colors in each image

__NOTE:__ We can't yet merge in the meta-information about what type of label we have until we include the img_num field into the img_meta table. Without it, we have no way to disambiguate.

In [24]:
df_colors.head()

,TTBID,b,g,img_num,percentage,r
0,16001001000009,57.854447,56.692722,0,0.50,60.746631
1,16001001000009,239.106518,238.742448,0,0.50,239.810811
2,16001001000018,166.517781,174.014225,0,0.50,179.844950
3,16001001000018,48.377104,50.838384,0,0.50,53.707071
4,16001001000018,232.319809,234.338902,1,0.25,236.873508


In [121]:
df_colors.groupby(['TTBID', 'img_num']).head()

,TTBID,b,g,img_num,percentage,r
0,16001001000009,57.854447,56.692722,0,0.500000,60.746631
1,16001001000009,239.106518,238.742448,0,0.500000,239.810811
2,16001001000018,166.517781,174.014225,0,0.500000,179.844950
3,16001001000018,48.377104,50.838384,0,0.500000,53.707071
4,16001001000018,232.319809,234.338902,1,0.250000,236.873508
5,16001001000018,98.923567,108.751592,1,0.250000,118.426752
6,16001001000018,165.893557,171.966387,1,0.250000,179.302521
7,16001001000018,14.328358,16.865672,1,0.250000,18.238806
8,16001001000018,9.704587,10.051376,2,0.500000,10.251376
9,16001001000018,73.578022,88.487912,2,0.500000,95.030769


In [25]:
num_colors = df_colors.groupby(['TTBID', 'img_num'])['percentage'].count()
num_colors = num_colors.unstack()

In [145]:
num_colors.columns

'0'

In [26]:
num_colors.head()

img_num,0,1,2,3,4,5,6,7
TTBID,,,,,,,,
16001001000009,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16001001000018,2.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN
16001001000021,2.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN
16001001000023,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
16001001000026,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
len(num_colors.columns)

6

In [28]:
data = []
for col in num_colors.columns:
    data.append(
        dict(
            type='histogram',
            histfunc='count',
            histnorm='percent',
            name='Img {}'.format(col),
            x=num_colors[col])
    )

layout = dict(
            title = 'Number of dominant colors detected in each image',
            xaxis = dict(title='Number of colors', tickangle = 0),
            yaxis = dict(title='Frequency')
             )

# clean version, no labels
#layout = dict(
#            xaxis = dict(visible = False),
#            yaxis = dict(visible = False)
#             )

fig = dict( data=data, layout=layout )
plotly.offline.iplot( fig )

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Popular colors

Find the popular color for each of our broad classes of drinks

##### Clean up the data

Notice that there are some extreme and invalid values in here

In [52]:
df_colors['r'].describe()

count    195973.000000
mean        155.808500
std          78.379905
min           0.000000
25%          92.393617
50%         169.175926
75%         228.836364
max         255.000000
Name: r, dtype: float64

Apply ceiling and floors to cap them out

In [51]:
df_colors[['r', 'g', 'b']] = df_colors[['r', 'g', 'b']].clip(lower=0.0, upper=255.0)

##### Discretize the color space

Calculate our bins and the labels we want associated with the bins

In [89]:
bins = np.linspace(0, 255, 5)
labels = (bins[1:] + bins[:-1]) / 2

Apply a `binned` categorical label to each of the rgb values

In [90]:
df_colors[['dr', 'dg', 'db']] = df_colors[['r', 'g', 'b']].apply(lambda x: pd.cut(x, bins, labels=labels, right=True, include_lowest=True))

In [91]:
df_colors.head()

,TTBID,b,g,img_num,percentage,r,dr,dg,db,str_color
0,16001001000009,57.854447,56.692722,0,0.50,60.746631,31.875,31.875,31.875,"rgb(70, 70, 70)"
1,16001001000009,239.106518,238.742448,0,0.50,239.810811,223.125,223.125,223.125,"rgb(240, 240, 240)"
2,16001001000018,166.517781,174.014225,0,0.50,179.844950,159.375,159.375,159.375,"rgb(184, 184, 155)"
3,16001001000018,48.377104,50.838384,0,0.50,53.707071,31.875,31.875,31.875,"rgb(42, 42, 42)"
4,16001001000018,232.319809,234.338902,1,0.25,236.873508,223.125,223.125,223.125,"rgb(240, 240, 240)"


In [92]:
df_colors['dr'].isnull().sum()

0

Create tuple strings, so we can more easily count the groupings of colors

In [93]:
df_colors['str_color'] = ['rgb({}, {}, {})'.format(int(r),int(g),int(b)) for (r,g,b) in zip(df_colors['dr'], df_colors['dg'], df_colors['db'])]

In [94]:
agg_colors = df_colors[['percentage', 'str_color']].groupby('str_color').sum()  # sum the percentage for each color combo
agg_colors.reset_index(inplace=True)  # bump str_color column back over
agg_colors.rename(columns={'percentage': 'agg_perc'}, inplace=True)  # rename percentage 

# add in the color channels individually
agg_colors = pd.merge(agg_colors, df_colors[['dr', 'dg', 'db', 'str_color']], left_on='str_color', right_on='str_color', how='left')
agg_colors.drop_duplicates(inplace=True)

#agg_colors[['dr', 'dg', 'db']] = agg_colors[['dr', 'dg', 'db']].apply(lambda x: x.cat.add_categories([0]))  # add 0 as a category
#agg_colors[['dr', 'dg', 'db']] = agg_colors[['dr', 'dg', 'db']].apply(lambda x: x.fillna(0))  # change NaN to 0

In [95]:
agg_colors['perc_scaled'] = agg_colors['agg_perc'] / agg_colors['agg_perc'].sum()

In [96]:
agg_colors.shape

(64, 6)

In [97]:
agg_colors.head()

,str_color,agg_perc,dr,dg,db,perc_scaled
0,"rgb(159, 159, 159)",7185.212037,159.375,159.375,159.375,0.128892
26975,"rgb(159, 159, 223)",207.106349,159.375,159.375,223.125,0.003715
27824,"rgb(159, 159, 31)",192.655688,159.375,159.375,31.875,0.003456
28679,"rgb(159, 159, 95)",1721.512302,159.375,159.375,95.625,0.030881
33966,"rgb(159, 223, 159)",134.723413,159.375,223.125,159.375,0.002417


In [98]:
agg_colors.sort_values('str_color', inplace=True)

In [99]:
data = []
for (scaled, color) in zip(agg_colors['perc_scaled'], agg_colors['str_color']):
    data.append(dict(
                    type = 'bar',
                    name = color,
                    x = [1],
                    y = [scaled],
                    marker = dict(color=color)))

layout = dict(
            barmode='stack')
    
# clean version, no labels
#layout = dict(
#            xaxis = dict(visible = False),
#            yaxis = dict(visible = False)
#             )

fig = dict( data=data, layout=layout )
plotly.offline.iplot( fig )

dr,14.1666666667,42.5,70.8333333333,99.1666666667,127.5,155.833333333,184.166666667,212.5,240.833333333
dp,,,,,,,,,
0.055556,617.0,581.0,562.0,693.0,755.0,828.0,839.0,1035.0,2040.0
0.166667,740.0,671.0,685.0,932.0,973.0,1096.0,1371.0,1462.0,2406.0
0.277778,552.0,616.0,600.0,623.0,759.0,907.0,1064.0,953.0,2195.0
0.388889,130.0,121.0,76.0,83.0,121.0,111.0,162.0,196.0,273.0
0.500000,649.0,1041.0,987.0,868.0,1044.0,1323.0,1071.0,950.0,3756.0
0.611111,123.0,105.0,60.0,55.0,63.0,107.0,196.0,214.0,278.0
0.722222,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
0.833333,59.0,11.0,4.0,5.0,2.0,10.0,25.0,32.0,42.0
0.944444,3.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
